In [11]:
from os import environ
import time
import numpy as np
import rasterio
from rasterio.windows import Window
from rasterio.plot import show
from rasterio import features
from affine import Affine
from rasterio.mask import mask as rmask
import geopandas as gpd
from geopandas import GeoDataFrame
from skimage.segmentation import slic, felzenszwalb, watershed
from skimage.future import graph
from skimage.morphology import disk
from skimage.filters.rank import modal
from skimage.measure import regionprops, label
from skimage.filters import sobel
from skimage.color import rgb2gray
from sklearn import svm
from collections import OrderedDict
import matplotlib
from matplotlib import pyplot
from rasterstats import zonal_stats
import pickle

# Analyzing Riparian Vegetation with GEOBIA
## by Aspen Manning

# Introduction

- Ephemeral streams
- Riparian vegetation
- Connectivity


![riparian vegetation](ChevelonRiparianVegetation.JPG)

# Study Area
Chevelon Fork, Northern Arizona
- Intermittent with ephemeral tributaries
- Tributary to Little Colorado River
- Landscape changes from headwaters to mouth



![Headwaters vegetation](ChevelonMtnUpland.jpg)

![Headwaters riparian vegetation](ChevelonPines.jpg)

![Mouth vegetation](ChevelonDryUpland.jpg)

![Desert Riparian vegetation](ChevelonTamarix.jpg)

# Goal for this project
![NDVI Map](NDVI.jpg)

# Methods

![Map 1](Mainmap.jpg)

# Segmentation and Attribution

In [ ]:
# from data_preparation.py as dp

def rasterize(gdf, value_column, shape, transform):
    """
    Rasterizes a GeoDataFrame
    
    Rasterizes a GeoDataFrame where the value_column becomes the pixel id.
    """
    p = _geometry_value_pairs(gdf, value_column)
    image = features.rasterize(p, out_shape=shape, transform=transform)
    
    return image

In [ ]:
# from data_preparation.py as dp

def vectorize(src=None, image=None, transform=None, crs=None):
    """
    Raster-to-Vector conversion.
    
    Performs a raster-to-vector conversion of a classified image. 
    """
    if src is not None:
        img = src.read(1, masked=True)
        transform = src.transform
        crs = src.crs.to_proj4()
    else:
        img = image[0].astype(np.int32)
        
    shps = features.shapes(img, transform=transform)
    records = []

    for id, shp in enumerate(shps):
        if shp[1] != 0:
            item = {'geometry': shp[0], 'id': id+1, 'properties': 
                    OrderedDict([('dn', np.int(shp[1]))]),
                    'type': 'Feature'}
            records.append(item)

    vec = GeoDataFrame.from_features(records)
    vec.crs = crs
    return vec

In [ ]:
# from data_preparation.py as dp

def add_zonal_properties(src=None, bands=[1,2,3], image=None, transform=None,
                         band_names=['red','green','blue'], stats=['mean'],
                         gdf=None):
    """
    Adds zonal properties to a GeoDataFrame.
    
    Adds zonal properties to a GeoDataFrame, where the statistics 'stats' are
    calculated for all pixels within the geographic objects boundaries.
     """
    if src is not None:
        image = src.read(bands, masked=True)
        transform = src.transform

    if len(image) != len(band_names): 
        print("The number of bands must equal the number of bands_names.")
        return None

    for band, name in enumerate(band_names):
        raster_stats = zonal_stats(gdf, image[band], stats=stats, 
                                   affine=transform)
        
        fields = [[] for i in range(len(stats))]
        labels = []
        
        for i, rs in enumerate(raster_stats):
            for j, r in enumerate(rs):
                if i == 0:
                    labels.append(r)
                fields[j].append(rs[r])
        
        for i, l in enumerate(labels):
            gdf.insert(len(gdf.columns)-1, name + "_" + l, fields[i])

    return gdf

In [ ]:
def get_prop(props, label):
    for p in props:
        if p.label == label:
            return p

In [ ]:
# from data_preparation.py as dp

def add_shape_properties(classified_image, gdf, attributes=['area', 'perimeter']):
    """
    Add raster properties as vector fields.
    """
    clim = classified_image[0, :, :]
    props = regionprops(clim)
    
    attributes = {s: [] for s in attributes}

    for row in gdf.itertuples():
        rid = getattr(row, 'dn')
        
        p = get_prop(props, rid)
        if p is not None:
            for a in attributes:
                attributes[a].append(getattr(p, a))

    try:
        for a in attributes:
            if (a == 'area'):
                gdf.insert(len(gdf.columns)-1, a, gdf.geometry.area)
            elif (a == 'perimeter'):
                gdf.insert(len(gdf.columns)-1, a, gdf.geometry.length)
            else:
                gdf.insert(len(gdf.columns)-1, a, attributes[a])
    except:
        print("The geometry is bad for this gdf.")
        print(gdf.columns)
    
    return gdf

In [ ]:
# from data_preparation.py as dp

def bsq_to_bip(image):
    # no error checking yet...
    return  image.transpose(1, 2, 0)


def bip_to_bsq(image):
    # no error checking yet...
    return  image.transpose(2, 0, 1)

In [ ]:
def sobel_edge_detect(src=None, band=1, image=None, mask=None):
    """
    Performs a Sobel edge detection.

    Performs a Sobel edge detection on a 2D image.
    """
    if src is not None:
        image = src.read(band, masked=True)
        mask = src.read_masks(1)
#         mask[mask > 0] = 1
    else:
        image = image
#         mask[mask > 255] = 1
        
    edges = sobel(image)
    return bip_to_bsq(edges[:, :, np.newaxis])

In [ ]:
# from data_preparation.py as dp

def segmentation(model=None, params=None, src=None, bands=[1,2,3], image=None,
                 mask=None, modal_radius=None, sieve_size=250):
    """
    Segment the image.

    Segment the image using an algorithm from sklearn.segmentation.
     """
    if src is not None:
        img = bsq_to_bip(src.read(bands, masked=True))
        mask = src.read_masks(1)
        mask[mask > 0] = 1
    else:
        img = bsq_to_bip(image)

    output = model(img, **params).astype('int32')

    while np.ndarray.min(output) < 1:
        output += 1

    if modal_radius != None:
        output = modal(output.astype('int16'), selem=disk(modal_radius))

    output = features.sieve(output, sieve_size)
    output = label(output, connectivity=1)
    
    output = bip_to_bsq(output[:, :, np.newaxis])

    return output

In [ ]:
# from skimage import data, io, segmentation, color
# from skimage.future import graph
# import numpy as np


def _weight_mean_color(graph, src, dst, n):
    diff = graph.node[dst]['mean color'] - graph.node[n]['mean color']
    diff = np.linalg.norm(diff)
    return {'weight': diff}


def merge_mean_color(graph, src, dst):
    graph.node[dst]['total color'] += graph.node[src]['total color']
    graph.node[dst]['pixel count'] += graph.node[src]['pixel count']
    graph.node[dst]['mean color'] = (graph.node[dst]['total color'] /
                                     graph.node[dst]['pixel count'])

In [ ]:
with rasterio.open("Chevelon.tif") as src:
    felz_params = {
        'scale': 0.01,
        'sigma': 0.5,
        'min_size': 10
    }

    ## This makes very small segments

In [ ]:
shapes = gpd.read_file("Chevelon.shp")
    out_image, out_transform = rmask(src, shapes.geometry, crop=True)
    image = out_image[0:3, :, :]
    rout = segmentation(model=felzenszwalb, params=felz_params, image=image)
    vout = vectorize(image=rout, transform=out_transform,
                     crs=src.crs.to_proj4())

    vout.to_file("ChevelonCkWS.gpkg", layer="initialSegments", driver="GPKG")
    orig = bsq_to_bip(image)
    labels = (bsq_to_bip(rout))[:, :, 0]

    rag = graph.rag_mean_color(orig, labels, mode='similarity', sigma=0.2)
    rout = graph.merge_hierarchical(labels, rag, thresh=50, rag_copy=False,
                             in_place_merge=True,
                             merge_func=merge_mean_color,
                             weight_func=_weight_mean_color)
    rout = bip_to_bsq(rout[:, :, np.newaxis])
    vout = vectorize(image=rout, transform=out_transform,
                    crs=src.crs.to_proj4())
    vout = add_zonal_properties(image=out_image, transform=out_transform, 
                                bands=[1, 2, 3, 4], band_names=['blue', 'green', 'red', 'nir', ],
                                stats=['mean','min','max','std'], gdf=vout)
    vout = add_shape_properties(rout, vout, ['area', 'perimeter',
                                             'eccentricity', 
                                             'equivalent_diameter',
                                             'major_axis_length',
                                             'minor_axis_length',
                                             'orientation'])
     edges = sobel_edge_detect(src, band=1)
    vout = add_zonal_properties(image=edges, band_names=['sobel'],
                                stats=['mean','min','max','std'],
                                transform=out_transform, gdf=vout)

    vout.to_file("ChevelonCkWS.gpkg", layer="ready2classify", driver="GPKG")

# Classification

In [ ]:
def train(X, Y):
    """
    Train classification algorithm.
    
    Train the Support Vector Machine classification algorithm using the
    specified fields. 
    """
    clf = svm.SVC()
        
    # specify parameters and distributions to sample from
    param_dist = {'C': expon(scale=100),
                  'gamma': expon(scale=.1),
                  'kernel': ['rbf'],
                  'class_weight':['balanced', None]}

    # run randomized search
    n_iter_search = 20
    random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search)

    random_search.fit(X, Y) # this may take time...
    
    return random_search

In [ ]:
def predict(model, X):
    """
    Classify segments using a trained SVM model

    Classify image segments using the trained Support Vector Machine model. 
    """
    predictions = model.predict(X)

    return predictions

In [ ]:
for_training = gpd.read_file("ClassifiedChevelon.gpkg")
for_training.head
big_train = for_training[~np.isnan(for_training["Class"])]
big_train.head
big_train.columns.values
labels = big_train['Class']
classes = big_train[['red_mean', 'green_mean', "blue_mean", "nir_mean", "eccentricity", "orientation", "sobel_max", "ndvi"]]
model = train(classes, labels)
to_predict = for_training[['red_mean', 'green_mean', "blue_mean", "nir_mean", "eccentricity", "orientation", "sobel_max", "ndvi"]]
output = predict(model, to_predict.values)
for_training['classified'] = output
for_training.to_file("chev2_output.gpkg", layer = "predictions", driver = "GPKG")

# Results
- 36,802 segments
- 3 classes: riparian vegetation, upland vegetation, bare soil/sparse vegetation
- 831 segments of riparian vegetation (mean NDVI = 0.47) 
- 3,630 segments of bare soil/sparse vegetation (mean NDVI = 0.20)
- 32,341 segments of upland vegetation (mean NDVI = 0.29) 

![Classified Map](Classes.jpg)

# Accuracy
- Checked against the NDVI map

![Accuracy map](Accuracy.jpg)

# Conclusions

- Classification would need tweaked to be useful 
- Segmentation is difficult when identifying small objects in a large image

# Thank you!